# Steam Workshop Downloader

Make sure you have `steamctl` installed. Use `pip install steamctl` to install it.

In [1]:
import os
import requests
from tqdm.notebook import tqdm, trange
from bs4 import BeautifulSoup

Settings

In [2]:
N_PAGES = 2  # number of pages to scrap
FILTER = 'spaceship'  # filter string
SORT = 'mostrecent'  # sorting order
APP_ID = 244850  # the Space Engineers App ID

API_KEY = ''
DOMAIN = ''

Create a directory to place the downloaded content.

In [3]:
os.makedirs('downloads', exist_ok=True)

Update the API_KEY in `steamctl`

In [4]:
with open('private', 'r') as f:
    ls = f.readlines()
    API_KEY = ls[0].strip().split(' ')[1]
    DOMAIN = ls[1].strip().split(' ')[1]

res = os.popen(f'steamctl webapi set-key {API_KEY}').read()
if not res.startswith('Current'):
    print('Error setting API key; check console.')

Scrape Workshop page to get the entries IDs

In [5]:
file_ids = []

for n in trange(N_PAGES, desc='Scraping workshop pages'):
    page_url = f'https://steamcommunity.com/workshop/browse/?appid={APP_ID}&searchtext={FILTER}&browsesort={SORT}&section=home&actualsort={SORT}&p={n}'
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text)
    entries = soup.find_all('a', class_='ugc')
    
    for entry in entries:
        file_ids.append(entry.get('data-publishedfileid'))

Scraping workshop pages:   0%|          | 0/2 [00:00<?, ?it/s]

Download each entry in its own folder

In [6]:
for i in trange(len(file_ids), desc='Downloading content'):
    file_id = file_ids[i]
    os.makedirs(f'downloads/{file_id}', exist_ok=True)
    os.popen(f'cd downloads/{file_id} && steamctl workshop download --anonymous {file_id}').read()